In [4]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [5]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///d:/RGT/Code/MLOps/Experiment-tracking/mlruns/2', creation_time=1704838014646, experiment_id='2', last_update_time=1704838014646, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='file:///d:/RGT/Code/MLOps/Experiment-tracking/mlruns/1', creation_time=1704485205099, experiment_id='1', last_update_time=1704485205099, lifecycle_stage='active', name='nyc-taxi-duration-prediction-experiment', tags={}>,
 <Experiment: artifact_location='file:///d:/RGT/Code/MLOps/Experiment-tracking/mlruns/0', creation_time=1704485205087, experiment_id='0', last_update_time=1704485205087, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris  
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():
    X, y = load_iris(return_X_y=True)
    
    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)
    
    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
    
    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")
    

2024/01/12 09:09:52 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: 'mlflow-artifacts:/3/84a01203329d455d84e2496417dacaeb/artifacts'


In [8]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1705079392905, experiment_id='3', last_update_time=1705079392905, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///d:/RGT/Code/MLOps/Experiment-tracking/mlruns/2', creation_time=1704838014646, experiment_id='2', last_update_time=1704838014646, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='file:///d:/RGT/Code/MLOps/Experiment-tracking/mlruns/1', creation_time=1704485205099, experiment_id='1', last_update_time=1704485205099, lifecycle_stage='active', name='nyc-taxi-duration-prediction-experiment', tags={}>,
 <Experiment: artifact_location='file:///d:/RGT/Code/MLOps/Experiment-tracking/mlruns/0', creation_time=1704485205087, experiment_id='0', last_update_time=1704485205087, lifecycle_stage='active', name='Default', tags={}>]

Interacting with the model registry

In [9]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")

In [10]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1704839432557, description=('Travel duration predictor models of NYC green taxi using regressors such as '
  'linear, lasso and ridge regression.\n'), last_updated_timestamp=1704840974096, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1704839703859, current_stage='None', description='', last_updated_timestamp=1704839703859, name='nyc-taxi-regressor', run_id='68fd832dbbac4fb0bec9cd416b9f9b12', run_link='', source='file:///d:/RGT/Code/MLOps/Experiment-tracking/mlruns/1/68fd832dbbac4fb0bec9cd416b9f9b12/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>,
  <ModelVersion: aliases=[], creation_timestamp=1704840655585, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2024-01-09', last_updated_timestamp=1704843589240, name='nyc-taxi-regressor', run_id='b6fc2de739c04b4bb936cbdfff80a106', run_link='', source='file:///d:/RGT/Code/MLOps/Experiment-tracking/

In [13]:
# Specify the experiment name or ID
experiment_name_or_id = '3'

# Get the run ID for the latest run in the specified experiment
runs = mlflow.search_runs(experiment_ids=experiment_name_or_id, order_by=["start_time desc"])
run_id = runs.iloc[0]['run_id']

# Register the model using the run ID
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/01/12 09:18:21 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1705079901699, current_stage='None', description='', last_updated_timestamp=1705079901699, name='iris-classifier', run_id='84a01203329d455d84e2496417dacaeb', run_link='', source='mlflow-artifacts:/3/84a01203329d455d84e2496417dacaeb/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>